In [223]:
import numpy as np
import matplotlib.pyplot as plt

In [224]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
input_str = '<' + input_str + '>'
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', '\'', '>']
print(f'{letters=}')
# print(f'{input_str=}')

len(letters)=32
502
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', "'", '>']


In [225]:
# generate SDR

SDR_len = 512 # 128
sparsity = 10 # 4
max_overlap = 1 # 1

sparse_prob = sparsity / SDR_len

letter_SDRs = {}

for letter in letters:
    while True:
        rnd_SDR = np.random.choice([0, 1], SDR_len, True, [1 - sparse_prob, sparse_prob])

        if (rnd_SDR > 0).sum() != sparsity:
            continue
        
        is_over_max = False
        for SDR in letter_SDRs.values():
            if (SDR & rnd_SDR).sum() > max_overlap:
                is_over_max = True
                break
        
        if is_over_max:
            continue
            
        break
    
    # print(f'{letter}')
    letter_SDRs.setdefault(letter, rnd_SDR)

for letter, SDR in letter_SDRs.items():
    print(f'{letter} : {(SDR > 0).sum()} {SDR.tolist()}')

a : 10 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [226]:
# text to SDRs
input_SDRs = []

for inp in input_str:
    input_SDRs.append(letter_SDRs[inp])

print(f'{len(input_SDRs)=}')

# for SDR in input_SDRs:
#     print(f'{SDR.tolist()}')

len(input_SDRs)=502


In [227]:
# init start column
column_size = 32

start_SDR = letter_SDRs['<']

start_I = np.zeros((start_SDR.size, column_size))

active_start_SDR_idx = np.where(start_SDR == 1)

for as_idx in active_start_SDR_idx[0]:
    rnd_active = np.random.randint(column_size)

    start_I[as_idx][rnd_active] = 1

# for col in np.rot90(start_I):
#     print(f'{col.astype(int).tolist()}')

In [228]:
from numpy import ndarray


sliding_window_view = np.lib.stride_tricks.sliding_window_view


def split_into_patches(I: ndarray, patch_width):
    left_pad = int(np.ceil(patch_width / 2))
    right_pad = int(np.floor(patch_width / 2))
    # print(left_pad, right_pad)
    pad_I = np.pad(I, ((left_pad, right_pad - 1), (0, 0)))

    # print(I.shape)
    v = sliding_window_view(x=pad_I, window_shape=patch_width, axis=0)
    # print(v.shape)
    # v = np.pad(v, (left_pad, right_pad))
    v = v.reshape(v.shape[0], 1, v.shape[1] * v.shape[2])
    # print(v.shape)
    
    return v

iI = np.arange(1, 17).reshape(8, 2)
print(iI)
split_into_patches(iI, 4)


[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]
 [11 12]
 [13 14]
 [15 16]]


array([[[ 0,  0,  1,  3,  0,  0,  2,  4]],

       [[ 0,  1,  3,  5,  0,  2,  4,  6]],

       [[ 1,  3,  5,  7,  2,  4,  6,  8]],

       [[ 3,  5,  7,  9,  4,  6,  8, 10]],

       [[ 5,  7,  9, 11,  6,  8, 10, 12]],

       [[ 7,  9, 11, 13,  8, 10, 12, 14]],

       [[ 9, 11, 13, 15, 10, 12, 14, 16]],

       [[11, 13, 15,  0, 12, 14, 16,  0]]])

In [229]:
input_patch_width = int(32 * 4)
print(f'{input_patch_width=}')

I = np.zeros((SDR_len, column_size))
O = np.zeros((SDR_len, column_size))

W = np.random.rand(SDR_len, column_size, column_size * input_patch_width) * 1e-2

V = np.zeros((SDR_len, column_size))

prec = 1e-30

# train

epoch_cnt = 1

eps0 = 0.003

eps0_V = 0.09

for epoch_idx in range(epoch_cnt):
    I = np.copy(start_I)

    for SDR in input_SDRs[1:]:
        rsh_I = split_into_patches(I=I, patch_width=input_patch_width)

        O = np.sum(W * rsh_I, axis=2)

        O /= np.max(np.abs(O)) + prec

        nV = np.abs(V - np.max(V, axis=1)[np.newaxis].T)
        nV /= np.max(nV, axis=1)[np.newaxis].T + prec

        OV = O + nV * eps0_V

        OV *= SDR[np.newaxis].T
        max_O = np.max(OV, axis=1)
        print(f'{(max_O > 0).sum()=}')

        O *= SDR[np.newaxis].T
        O[OV < max_O[np.newaxis].T] = 0
        max_O_idx = np.where(O != 0)
        print(f'{max_O_idx[0].size=}')

        # print(max_O_idx[0], rsh_I.shape)
        rsh_I = rsh_I + np.zeros(W.shape)
        # print(rsh_I.shape)
        W[max_O_idx] += (rsh_I[max_O_idx] - W[max_O_idx] * O[max_O_idx][..., np.newaxis]) * eps0

        # winners
        V[max_O_idx] += 1

        # recur
        O[O != 0] = 1

        I = np.copy(O)

# O.shape
print(np.mean(V, axis=1).tolist())
print(np.sum(V == 0))
print(np.sum((V > 0), axis=1).tolist())
# for col in np.rot90(V):
#     print(f'{col.astype(int).tolist()}')

input_patch_width=128


(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=7
max_O_idx[0].size=7
(max_O > 0).sum()=6
max_O_idx[0].size=6
(max_O > 0).sum()=9
max_O_idx[0].size=9
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=9
max_O_idx[0].size=9
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=6
(max_O > 0).sum()=10
max_O_idx[0].size=7
(max_O > 0).sum()=10
max_O_idx[0].size=8
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=8
(max_O > 0).sum()=10
max_O_idx[0].size=9
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=9
(max_O > 0).sum()=10
max_O_idx[0].size=10
(max_O > 0).sum()=10
max_O_idx[0].size=9
(max_O 

In [230]:
# test
error = []

I = np.copy(start_I)

for SDR in input_SDRs[1:]:
    rsh_I = split_into_patches(I=I, patch_width=input_patch_width)

    O = np.sum(W * rsh_I, axis=2)

    max_O = np.max(O, axis=1)
    argmax_O_idx = np.argsort(max_O)[::-1][:sparsity]

    SDR_O = np.zeros(O.shape[0])
    SDR_O[argmax_O_idx] = 1

    err = (SDR.astype(int) ^ SDR_O.astype(int)).sum()
    # print(f'{err=}')
    error.append(err)

    O *= SDR[np.newaxis].T
    max_O = np.max(O, axis=1)
    O[O < max_O[np.newaxis].T] = 0

    # recur
    O[O != 0] = 1

    I = np.copy(O)

error = np.array(error)
mean_percent = int(np.mean(error) / (2 * sparsity) * 100)
print(f'{np.mean(error)=} {error.max()=} {error.min()=} {mean_percent=}')

# O.shape
# for col in np.rot90(O):
#     print(f'{col.astype(int).tolist()}')

np.mean(error)=16.686626746506985 error.max()=20 error.min()=8 mean_percent=83
